In [1]:
# libraries
import os
import json
import numpy as np
import pandas as pd
from extra_files import helper as hp
from imageio import imwrite, imread
from skimage.transform import resize

In [2]:
# File paths
data_path = '/home/aldo/Documents/data-cic/'
preprocess_path = data_path + 'preprocess_data'

## Training SSDLite300

In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssdlite320_mobilenetv2 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files.f1_callback import F1_callback as f1_call
%matplotlib inline

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


### Parameters (original SSD300 architecture)

In [4]:
## Parameteres needed for ssd_300() and SSDInputEncoder()

img_height = 320 # Height of the model input images
img_width = 320 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [-1., -1., -1.] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
divide_by_stddev = [127.5, 127.5, 127.5]
swap_channels = False # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
new_scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
scales = new_scales
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [16, 32, 64, 107, 160, 320] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

## Create new model with SSD weights

In [5]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                divide_by_stddev=divide_by_stddev,
                swap_channels=swap_channels)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
# weights_path = '../weights/models/ssdlite320_mobilenetv1/ssdlite320_mobilenetv1_pascal.h5'
# model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [6]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
identity_layer (Lambda)         (None, 320, 320, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
input_stddev_normalization (Lam (None, 320, 320, 3)  0           identity_layer[0][0]             
__________________________________________________________________________________________________
input_mean_normalization (Lambd (None, 320, 320, 3)  0           input_stddev_normalization[0][0] 
__________________________________________________________________________________________________
model_1 (M

## Data generator for the training

In [7]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
images_dir = data_path + 'pascal_dataset'
# images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
train_labels_filename = preprocess_path + '/pascal_train.csv'
val_labels_filename   = preprocess_path + '/pascal_val.csv'
# train_labels_filename = preprocess_path + '/cic_train.csv'
# val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 2097/2097 [00:08<00:00, 260.88it/s]
Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


In [8]:
# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv13_mbox_conf').output_shape[1:3],
                   model.get_layer('conv17_mbox_conf').output_shape[1:3],
                   model.get_layer('conv18_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv19_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv20_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv21_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	  6469
Number of images in the validation dataset:	  2097


## Remaining training parameters

In [9]:
label_val = np.load('../data-cic/preprocess_data/label_val_320.npy')
val_images_320 = np.load('../data-cic/preprocess_data/images_val_320x320.npy')

In [10]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)

csv_logger = CSVLogger(filename=data_path + 'history/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal.csv',
                       separator=',',
                       append=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=12,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.5,
                                         patience=10,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.000001)

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_320, 
                      label_val, (1, 2424, 14),
                      data_path + 'history/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_f1.csv')

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule, verbose=1)


callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
#              learning_rate_scheduler]
             reduce_learning_rate,
             f1_callback]

In [11]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 1000
steps_per_epoch = 400

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/1000
400/400 [==============================] - 229s 573ms/step - loss: 5.6792 - val_loss: 4.5497

Epoch 00001: val_loss improved from inf to 4.54973, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal.h5
Number of images: 45
Presicion: 0.2355
Recall: 0.3721
F1 score: 0.2885
Improve F1 score from -inf to 0.2884544922710439
Epoch 2/1000
400/400 [==============================] - 221s 552ms/step - loss: 4.3143 - val_loss: 5.6021

Epoch 00002: val_loss did not improve from 4.54973
Number of images: 45
Presicion: 0.1656
Recall: 0.4336
F1 score: 0.2396
Epoch 3/1000
400/400 [==============================] - 221s 552ms/step - loss: 3.8307 - val_loss: 4.6025

Epoch 00003: val_loss did not improve from 4.54973
Number of images: 45
Presicion: 0.3395
Recall: 0.384
F1 score: 0.3604
Improve F1 score from 0.2884544922710439 to 0.3603616046773314
Epoch 4/1000
400/400 [==============================] - 221s 553ms/step - loss: 3.6705 - val_

400/400 [==============================] - 219s 548ms/step - loss: 2.9924 - val_loss: 3.0219

Epoch 00030: val_loss did not improve from 3.01753
Number of images: 45
Presicion: 0.5192
Recall: 0.3454
F1 score: 0.4148
Epoch 31/1000
400/400 [==============================] - 220s 550ms/step - loss: 2.9774 - val_loss: 3.1560

Epoch 00031: val_loss did not improve from 3.01753
Number of images: 45
Presicion: 0.4692
Recall: 0.4026
F1 score: 0.4334
Epoch 32/1000
400/400 [==============================] - 221s 552ms/step - loss: 2.9821 - val_loss: 3.4688

Epoch 00032: val_loss did not improve from 3.01753
Number of images: 45
Presicion: 0.4384
Recall: 0.2738
F1 score: 0.3371
Epoch 33/1000
400/400 [==============================] - 224s 561ms/step - loss: 2.9522 - val_loss: 3.2251

Epoch 00033: val_loss did not improve from 3.01753
Number of images: 45
Presicion: 0.5274
Recall: 0.3918
F1 score: 0.4496
Improve F1 score from 0.44646350453412564 to 0.44959720659224206
Epoch 34/1000
400/400 [======


Epoch 00091: val_loss did not improve from 2.60846
Number of images: 45
Presicion: 0.5797
Recall: 0.4709
F1 score: 0.5197
Epoch 92/1000
400/400 [==============================] - 214s 535ms/step - loss: 2.4041 - val_loss: 2.6115

Epoch 00092: val_loss did not improve from 2.60846
Number of images: 45
Presicion: 0.5634
Recall: 0.4667
F1 score: 0.5105
Epoch 93/1000
400/400 [==============================] - 215s 538ms/step - loss: 2.3999 - val_loss: 2.6167

Epoch 00093: val_loss did not improve from 2.60846
Number of images: 45
Presicion: 0.6025
Recall: 0.4812
F1 score: 0.5351
Improve F1 score from 0.5308118833880706 to 0.5350965757554014
Epoch 94/1000
400/400 [==============================] - 215s 538ms/step - loss: 2.4187 - val_loss: 2.6230

Epoch 00094: val_loss did not improve from 2.60846
Number of images: 45
Presicion: 0.5945
Recall: 0.4841
F1 score: 0.5337
Epoch 95/1000
400/400 [==============================] - 215s 537ms/step - loss: 2.4023 - val_loss: 2.5995

Epoch 00095: val

Number of images: 45
Presicion: 0.6068
Recall: 0.4809
F1 score: 0.5366
Epoch 123/1000
400/400 [==============================] - 216s 539ms/step - loss: 2.3094 - val_loss: 2.5892

Epoch 00123: val_loss did not improve from 2.58838
Number of images: 45
Presicion: 0.6101
Recall: 0.4756
F1 score: 0.5345
Epoch 00123: early stopping


In [12]:
# Reset learning rate to 0.001
adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [13]:
# 1: Instantiate two `DataGenerator` objects: One for training, one for validation.

# Optional: If you have enough memory, consider loading the images into memory for the reasons explained above.

train_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)
val_dataset = DataGenerator(load_images_into_memory=True, hdf5_dataset_path=None)

# 2: Parse the image and label lists for the training and validation datasets.

# TODO: Set the paths to your dataset here.

# Images
# images_dir = data_path + 'udacity_driving_datasets'
# images_dir = data_path + 'pascal_dataset'
images_dir = data_path + 'images'

# Ground truth
# train_labels_filename = preprocess_path + '/udc.csv'
# train_labels_filename = preprocess_path + '/cic_pascal_train.csv'
# val_labels_filename   = preprocess_path + '/cic_pascal_val.csv'
# train_labels_filename = preprocess_path + '/pascal_train.csv'
# val_labels_filename   = preprocess_path + '/pascal_val.csv'
train_labels_filename = preprocess_path + '/cic_train.csv'
val_labels_filename   = preprocess_path + '/cic_val.csv'

train_dataset.parse_csv(images_dir=images_dir,
                        labels_filename=train_labels_filename,
                        input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'], # This is the order of the first six columns in the CSV file that contains the labels for your dataset. If your labels are in XML format, maybe the XML parser will be helpful, check the documentation.
                        include_classes='all')

val_dataset.parse_csv(images_dir=images_dir,
                      labels_filename=val_labels_filename,
                      input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                      include_classes='all')

# Optional: Convert the dataset into an HDF5 dataset. This will require more disk space, but will
# speed up the training. Doing this is not relevant in case you activated the `load_images_into_memory`
# option in the constructor, because in that cas the images are in memory already anyway. If you don't
# want to create HDF5 datasets, comment out the subsequent two function calls.

# train_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_train.h5',
#                                   resize=False,
#                                   variable_image_size=True,
#                                   verbose=True)

# val_dataset.create_hdf5_dataset(file_path='dataset_udacity_traffic_val.h5',
#                                 resize=False,
#                                 variable_image_size=True,
#                                 verbose=True)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Loading images into memory: 100%|██████████| 45/45 [00:01<00:00, 28.01it/s]
Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [14]:
# 3: Set the batch size.
batch_size = 32 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.
# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.
# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv13_mbox_conf').output_shape[1:3],
                   model.get_layer('conv17_mbox_conf').output_shape[1:3],
                   model.get_layer('conv18_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv19_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv20_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv21_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)

val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,
                                                      resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)

# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	   210
Number of images in the validation dataset:	    45


In [15]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
# model_checkpoint.best = 2.567152314715915

csv_logger = CSVLogger(filename=data_path + 'history/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.csv',
                       separator=',',
                       append=True)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=12,
                               verbose=1)

reduce_learning_rate = ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.5,
                                         patience=10,
                                         verbose=1,
                                         min_delta=0.001,
                                         cooldown=0,
                                         min_lr=0.000001)

f1_callback = f1_call(0.20, 
                      0.45, 
                      200, 
                      normalize_coords, 
                      img_height, 
                      img_width, 
                      val_images_320, 
                      label_val, (1, 2424, 14),
                      data_path + 'history/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic_f1.csv')

# learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule, verbose=1)


callbacks = [model_checkpoint,
             csv_logger,
             early_stopping,
#              learning_rate_scheduler]
             reduce_learning_rate,
             f1_callback]

In [16]:
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 1000
steps_per_epoch = 15

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

Epoch 1/1000
15/15 [==============================] - 66s 4s/step - loss: 3.3283 - val_loss: 2.9904

Epoch 00001: val_loss improved from inf to 2.99039, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.h5
Number of images: 45
Presicion: 0.6378
Recall: 0.5186
F1 score: 0.5721
Improve F1 score from -inf to 0.5720661748345818
Epoch 2/1000
15/15 [==============================] - 61s 4s/step - loss: 2.8354 - val_loss: 2.8278

Epoch 00002: val_loss improved from 2.99039 to 2.82776, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.h5
Number of images: 45
Presicion: 0.645
Recall: 0.5363
F1 score: 0.5856
Improve F1 score from 0.5720661748345818 to 0.5856417903961293
Epoch 3/1000
15/15 [==============================] - 64s 4s/step - loss: 2.8889 - val_loss: 2.8025

Epoch 00003: val_loss improved from 2.82776 to 2.80247, saving model to /home/aldo/Documents/weights/models/ssdl

15/15 [==============================] - 63s 4s/step - loss: 2.1317 - val_loss: 2.5701

Epoch 00031: val_loss improved from 2.61244 to 2.57007, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.h5
Number of images: 45
Presicion: 0.7376
Recall: 0.4901
F1 score: 0.5889
Epoch 32/1000
15/15 [==============================] - 63s 4s/step - loss: 1.9586 - val_loss: 2.5910

Epoch 00032: val_loss did not improve from 2.57007
Number of images: 45
Presicion: 0.7135
Recall: 0.5072
F1 score: 0.5929
Epoch 33/1000
15/15 [==============================] - 61s 4s/step - loss: 2.0320 - val_loss: 2.5991

Epoch 00033: val_loss did not improve from 2.57007
Number of images: 45
Presicion: 0.7174
Recall: 0.471
F1 score: 0.5686
Epoch 34/1000
15/15 [==============================] - 60s 4s/step - loss: 1.9874 - val_loss: 2.5062

Epoch 00034: val_loss improved from 2.57007 to 2.50616, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobil

Number of images: 45
Presicion: 0.6959
Recall: 0.4894
F1 score: 0.5746
Epoch 63/1000
15/15 [==============================] - 60s 4s/step - loss: 1.7863 - val_loss: 2.4042

Epoch 00063: val_loss improved from 2.42096 to 2.40416, saving model to /home/aldo/Documents/weights/models/ssdlite320_mobilenetv2/ssdlite320_mobilenetv2_pascal_cic.h5
Number of images: 45
Presicion: 0.6928
Recall: 0.5699
F1 score: 0.6254
Epoch 64/1000
15/15 [==============================] - 63s 4s/step - loss: 1.9159 - val_loss: 2.5404

Epoch 00064: val_loss did not improve from 2.40416
Number of images: 45
Presicion: 0.7006
Recall: 0.4934
F1 score: 0.579
Epoch 65/1000
15/15 [==============================] - 63s 4s/step - loss: 1.8836 - val_loss: 2.5233

Epoch 00065: val_loss did not improve from 2.40416
Number of images: 45
Presicion: 0.7046
Recall: 0.5192
F1 score: 0.5979
Epoch 66/1000
15/15 [==============================] - 60s 4s/step - loss: 1.7918 - val_loss: 2.4503

Epoch 00066: val_loss did not improve f